# Prometheus

In [ ]:
%load_ext autoreload
%autoreload 2

from prometheus import Prometheus_Client
from chaos import Chaos
import dateparser
import pytz
import yaml

In [ ]:
prom = Prometheus_Client(url="http://localhost:9092")
namespace = 'default'
pods = prom.get_pod_names(namespace=namespace)
pods = [x for x in pods if not x.startswith("otelcollector") and not x.startswith("jaeger")]
pods
# prom.get_instance(type="mysql")

In [ ]:
pod = pods[0]
print(pod)
chaos = Chaos()
# chaos.delete()
experiment = "network-delay-target-serial"
template_path= "../chaos_experiment/templates/Serial/{}.yaml".format(experiment)
chaos_path = "../chaos_experiment/experiments/Serial/{}-{}-{}.yaml".format(experiment,namespace,pod)
chaos.load(template_path)
chaos.execute(f_path=chaos_path,namespace=namespace)
# chaos_stats = chaos.status(kind='NetworkChaos',name='network-delay-serial',namespace=namespace)


In [ ]:
query_f_path = "./test.txt"
# start_time = parse_datetime("10min")
# end_time = parse_datetime("now")
tz = "Asia/Shanghai"

chaos_creation_time = chaos.creation_time.astimezone(pytz.timezone(tz))
print(chaos_creation_time)
start_time = dateparser.parse("5 min ago",settings={"RELATIVE_BASE":chaos_creation_time})
chaos_endtime = dateparser.parse("in %s" % (chaos.duration),settings={"RELATIVE_BASE":chaos_creation_time})
print(chaos_endtime)
end_time = dateparser.parse("in 5 min",settings={"RELATIVE_BASE":chaos_endtime})

step = str(60 * 0.5)  # 60 * 20  20 minutes

In [ ]:
f = open('../config/METRIC.yaml')
data = f.read()
f.close()
queries = yaml.safe_load(data)

In [ ]:
# query_type = 'standard'
query_type = 'custom'
metric_type = 'icmp'
query = queries[query_type][metric_type][0]['query']
idx = queries[query_type][metric_type][0]['index']

In [ ]:
if query_type == 'standard':
    query= query.replace("$namespace", namespace).replace("$pod", pod)
elif query_type == 'custom':
    if metric_type == 'icmp':
        query = query.replace("$pod", pod.split('-')[0] + '-icmp')

query

In [ ]:
metric = prom.query_metric(
                query,
                idx,
                start_time=start_time,
                end_time=end_time,
                step=step,
                namespace=namespace,
                pod=pod,
                chaos=chaos,
                save=True,
            )

In [ ]:
prom.plot_metric(chaos=chaos,namespace=namespace,pod=pod,idx=idx,tz=tz)

# Jaeger

In [ ]:
%load_ext autoreload
%autoreload 2
from jaeger import Jaeger_Client

jaeger = Jaeger_Client(url="http://localhost:16680")

In [ ]:
jaeger.write_traces_all(service="adservice")
jaeger.get_traces(service="adservice", limit=5)